In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn import linear_model

data = pd.read_csv("online_classroom.csv")


In [2]:
target = 'falsexam'
treatment = 'format_ol'

In [3]:
# Efecto a primera vista
data.query("format_ol == 1").falsexam.mean()

73.63526308510637

In [4]:
data.query("format_ol != 1").falsexam.mean()

77.85552344978166

Por propiedades de la regresión lineal (que no veremo en este curso de momento) las $\beta_i$ son realmente un promedio de las variables categoricas.

Quiere decir que podríamos usar una regresión para probar hipotesis

In [7]:
# Regresion
model = smf.ols("falsexam ~ format_ol", data=data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               falsexam   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     8.938
Date:                Tue, 11 Feb 2025   Prob (F-statistic):            0.00301
Time:                        20:12:07   Log-Likelihood:                -1246.9
No. Observations:                 323   AIC:                             2498.
Df Residuals:                     321   BIC:                             2505.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     77.8555      0.762    102.235      0.0

El modelo nos da el coeficiente, que es exactamente igual a la diferencia de los promedios

#### Intentando estimar la variabilidad de $\beta$, resampleo

In [51]:
np.std(coeficiente)

1.6385911479498296

#### Varianza de los residuos
$\hat{\sigma^2} = \frac{\sum(y - \hat{y})^2}{n - p - 1}$

Donde:

$n$  es el total de datos

$p$ total de variables

Esto sería simplemente el error cuadrático promedio de mi modelo 

#### Variabilidad de lo que usas para predecir 

Suma de los cuadrados de X  =$\sum (x - \hat{x_i})^2$

Esto es la varianza de tu predictor pero sin dividir entre n

#### Varianza del coeficiente $\beta$

$SE(\beta)$ = $\sqrt{\frac{\hat{\sigma^2}}{\sum (x - \hat{x_i})^2}}$


### Estadístico t

$t = \lvert \frac{\beta_i}{SE(\beta_i)} \rvert$

nos dice cuántas desviaciones estándar $\beta$ está  lejos de cero.

Ojo: la distibución $t$ es simetrica y puede ser del lado derecho o izquierdo, habría que multiplicar este p value por dos



P - Value

$2 * P(T > \lvert t \rvert)$

Si bien el estadístico t nos dice cuantas desviaciones estandard está por arriba de 0, puede ser a cualquier lado 

- Si t es muy positivo, queremos ver la probabilidad de obtener un valor tan alto o mayor.

- Si t es muy negativo, queremos ver la probabilidad de obtener un valor tan bajo o menor.

P-value

1 - stats.t.cdf(abs(t), df=len(data))

Intervalos de confianza

### Usando stats model 

Sklearn no nos da el intervalo de confianza ni el valor P de los estimados, afortunadamente statsmodel si nos lo puede dar

### Volviendo al ejemplo de las casas

median_income y total_rooms

In [95]:
df = pd.read_csv('housing.csv')